In [196]:
import requests
from l402_types import L402Info


### Getting L402 URI Info

In [199]:
def get_l402_uri_info(uri: str) -> dict:
    """
   Returns the information of the L402 URI resource.

    Args:
    uri (str): The L402 URI to describe.

    Returns:
    dict: A dictionary containing the description of the L402 URI.
    """
    if not uri.startswith("l402://"):
        raise ValueError("Invalid L402 URI format")
    
    scheme = "http://" if "localhost" in uri else "https://"
    http_url = uri.replace("l402://", scheme, 1)
    
    response = requests.get(http_url)
    response.raise_for_status()
    
    return response.json()

    

In [94]:
info = get_l402_uri_info("l402://localhost:8080/v0/gateway/634ae965-03db-4bb7-872e-c23dd1b8d75f/info")
# info = get_l402_uri_info("l402://blockbuster.fewsats.com/video/info/79c816f77fdc4e66b8cd18ad67537936")
info['access'].pop('authentication', None)
info

{'access': {'endpoint': 'http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f',
  'method': 'POST'},
 'content_type': 'api',
 'cover_url': '',
 'description': 'Scrape a given URL. Pass the URL as JSON in the request body as follows:\n```\n{"url": url }\n```',
 'name': 'Web Scraping\n',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '1.0'}

### Generating Python Function

In [248]:
from cosette import Chat
import json 
func_generation_sp = """You are an AI assistant specialized in creating Python functions based on L402 info inputs. 
When given an L402 info dictionary, your task is to generate a Python function that can access the specified endpoint. 
Follow these guidelines:

1. Create a function name that relates to the resource or action described in the 'name' field making it specific enough to avoid conflicts.
2. Use the 'endpoint' field to determine the URL for the request.
3. Use the 'method' field to determine the HTTP method for the request.
4. Handle any required parameters for the endpoint request by passing them as function params.
5. Write a docstring that includes:
   - A brief description of the function's purpose
   - Parameters with their types and descriptions
   - Return value with its type and description
6. Do not import the requests library. Assume it is available in the global scope.
7. Handle potential errors and exceptions appropriately.
8. Do not handle any authentication or authorization in the function.
9. Return the response from the endpoint.
10. Use L402 request client to send the request and configure exactly like this:

L402 Request configuration:
```
l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)
```

Example input:
```
{'access': {'authentication': {'format': 'L402 {credentials}:{proof_of_payment}',
   'header': 'Authorization',
   'protocol': 'L402'},
  'endpoint': 'https://blockbuster.fewsats.com/video/stream/79c816f77fdc4e66b8cd18ad67537936',
  'method': 'POST'},
 'content_type': 'video',
 'cover_url': 'https://pub-3c55410f5c574362bbaa52948499969e.r2.dev/cover-images/79c816f77fdc4e66b8cd18ad67537936',
 'description': 'Lex Fridman Podcast full episode:    • Andrew Huberman: Focus, Controversy, ',
 'name': 'How to focus and think deeply | Andrew Huberman and Lex Fridman',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '1.0'}
```

Example output:
```
from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)
 def stream_video_huberman() -> dict:
    \"\"\"
    Stream the video of the podcast episode featuring Andrew Huberman and Lex Fridman.

    This function sends a POST request to the specified endpoint.
    Returns:
    - requests.Response: The response object containing the server's response to the request.
    \"\"\"
    endpoint = "https://blockbuster.fewsats.com/video/stream/79c816f77fdc4e66b8cd18ad67537936"
    try:
        response = l402_requests.post(endpoint, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
```
Your response should be a complete Python function definition, including imports, type hints, docstring, and function body. Only output the function definition, no other text."""


In [249]:
def generate_python_function(info: L402Info) -> str:
   """
   Generates a Python function based on the provided L402 info dictionary.

   Args:
   info (L402Info): The L402 info dictionary containing the details of the resource.

   Returns:
   str: A Python function definition as a string.
   """

   chat = Chat('gpt-4o-mini', sp=func_generation_sp)
   return chat(json.dumps(info))

func_code = generate_python_function(info)

### Extracting the function code

In [ ]:
def get_text(response):
    return response.choices[0].message.content

response = get_text(func_code)
response

In [180]:
import re
def extract_function_code(func_code):
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(func_code)
    return match.group(1).strip() if match else func_code


In [ ]:
import re
def extract_function_code(func_code):
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(func_code)
    return match.group(1).strip() if match else func_code


In [181]:
extracted_code = extract_function_code(response)
print(extracted_code)

from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)

def scrape_web_url(url: str) -> requests.Response:
    """
    Scrape a given URL by sending a POST request with the URL in the request body.

    This function sends a POST request to the specified endpoint with the URL parameter included in the body as JSON.
    
    Parameters:
    - url (str): The URL to scrape.

    Returns:
    - requests.Response: The response object containing the server's response to the request.
    """
    endpoint = "http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f"
    json_data = {"url": url}
    try:
        response = l402_requests.post(endpoint, json=json_data)
        response.raise_for_status()  # Ra

### Creating the function

In [ ]:
import importlib.util
import re
import os

def create_func(code_string):
    # Extract Python code from Markdown code blocks if present
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(code_string)
    extracted_code = match.group(1).strip() if match else code_string

    # Create a temporary file name
    temp_file = 'temp_function.py'

    # try:
        # Write the code to a temporary file
    with open(temp_file, 'w') as f:
        f.write(extracted_code)

    # Import the function
    spec = importlib.util.spec_from_file_location("temp_module", temp_file)
    temp_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(temp_module)

    # Get the function
    function_name = extracted_code.split('def ')[1].split('(')[0].strip()
    return getattr(temp_module, function_name)

    # finally:
    #     # Remove the temporary file
    #     if os.path.exists(temp_file):
    #         os.remove(temp_file)

In [183]:

gf = create_func(extracted_code)


In [188]:
r = gf('https://www.spacex.com/')
r = r.json()

<Response [200]>

In [194]:
from IPython.display import display, Markdown
display(Markdown(r['data']['markdown']))


UPCOMING LAUNCH

## Starship's Fifth Flight Test

[LEARN MORE](/launches/mission/?missionId=starship-flight-5)

Recent Launch

## Hera Mission

[Rewatch](/launches/mission/?missionId=hera)

CURRENT MISSION

## CREW-9 MISSION

[REWATCH](/launches/mission/?missionId=crew-9)

RECENT MISSION

## POLARIS DAWN

[REWATCH](/launches/mission/?missionId=polarisdawn)

## ADVANCING HUMAN SPACEFLIGHT

[LEARN MORE](/updates/#eva-suit)

## TO MAKE LIFE MULTIPLANETARY

[LEARN MORE](/updates/#make-life-multiplanetary)

## Adding Tools to the Agent

In [268]:
tools = [get_l402_uri_info]
chat = Chat('gpt-4o-mini', sp='You are a helpful assistant that provides information about L402 URIs and your available tools.', tools=tools)

chat("which tools do you have available?")

I have access to a tool that provides information about L402 URIs. Specifically, I can retrieve the description of a given L402 URI using the tool.

<details>

- id: chatcmpl-AGTocC7ldgZPEKheNTAoEq52fhRHm
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I have access to a tool that provides information about L402 URIs. Specifically, I can retrieve the description of a given L402 URI using the tool.', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728491222
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=33, prompt_tokens=111, total_tokens=144, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [270]:
r = chat('What is the info of: l402://localhost:8080/v0/gateway/634ae965-03db-4bb7-872e-c23dd1b8d75f/info')
display(Markdown(get_text(r)))

Here is the information for the L402 URI `l402://localhost:8080/v0/gateway/634ae965-03db-4bb7-872e-c23dd1b8d75f/info`:

- **Name**: Web Scraping
- **Description**: Scrape a given URL. Pass the URL as JSON in the request body as follows:
  ```
  {"url": url }
  ```
- **Version**: 1.0
- **Content Type**: api
- **Pricing**: 
  - Amount: 1
  - Currency: USD
- **Access**:
  - **Authentication**:
    - Format: L402 {credentials}:{proof_of_payment}
    - Header: Authorization
    - Protocol: L402
  - **Endpoint**: http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f
  - **Method**: POST
- **Cover URL**: (not provided)

In [271]:
uri="l402://localhost:8080/v0/gateway/634ae965-03db-4bb7-872e-c23dd1b8d75f/info"

# Step 1: Get L402 URI info
info = get_l402_uri_info(uri)
print("L402 URI info retrieved successfully.")
info['access'].pop('authentication', None)
info


L402 URI info retrieved successfully.


{'access': {'endpoint': 'http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f',
  'method': 'POST'},
 'content_type': 'api',
 'cover_url': '',
 'description': 'Scrape a given URL. Pass the URL as JSON in the request body as follows:\n```\n{"url": url }\n```',
 'name': 'Web Scraping\n',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '1.0'}

In [272]:
# Step 2: Generate Python function
func_code = generate_python_function(info)
print("Python function generated successfully.")
func_code

Python function generated successfully.


```python
from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)

def scrape_web_url(url: str) -> dict:
    """
    Scrape a given URL by sending a POST request to the specified endpoint.
    
    Parameters:
    - url (str): The URL to be scraped, which will be passed as JSON in the request body.
    
    Returns:
    - dict: The response object containing the server's response to the request.
    """
    endpoint = "http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f"
    payload = {"url": url}
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = l402_requests.post(endpoint, json=payload, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
```

<details>

- id: chatcmpl-AGTplsuoyWGGdlE5aQANUk9EYIRIk
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```python\nfrom l402.client import requests as l402_requests\nfrom l402.client.preimage_provider import AlbyAPI\nfrom l402.client.credentials import SqliteCredentialsService\nimport requests\nimport os\n\nl402_requests.configure(\n   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),\n   credentials_service=SqliteCredentialsService()\n)\n\ndef scrape_web_url(url: str) -> dict:\n    """\n    Scrape a given URL by sending a POST request to the specified endpoint.\n    \n    Parameters:\n    - url (str): The URL to be scraped, which will be passed as JSON in the request body.\n    \n    Returns:\n    - dict: The response object containing the server\'s response to the request.\n    """\n    endpoint = "http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f"\n    payload = {"url": url}\n    headers = {\'Content-Type\': \'application/json\'}\n    \n    try:\n        response = l402_requests.post(endpoint, json=payload, headers=headers)\n        response.raise_for_status()  # Raise an error for bad responses\n        return response.json()\n    except requests.exceptions.RequestException as e:\n        print(f"An error occurred: {e}")\n        return None\n```', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728491293
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=276, prompt_tokens=889, total_tokens=1165, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [273]:

# Step 3: Extract function code
extracted_code = extract_function_code(get_text(func_code))
print("Function code extracted successfully.")
extracted_code

Function code extracted successfully.


'from l402.client import requests as l402_requests\nfrom l402.client.preimage_provider import AlbyAPI\nfrom l402.client.credentials import SqliteCredentialsService\nimport requests\nimport os\n\nl402_requests.configure(\n   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),\n   credentials_service=SqliteCredentialsService()\n)\n\ndef scrape_web_url(url: str) -> dict:\n    """\n    Scrape a given URL by sending a POST request to the specified endpoint.\n    \n    Parameters:\n    - url (str): The URL to be scraped, which will be passed as JSON in the request body.\n    \n    Returns:\n    - dict: The response object containing the server\'s response to the request.\n    """\n    endpoint = "http://localhost:8080/v0/gateway/access/634ae965-03db-4bb7-872e-c23dd1b8d75f"\n    payload = {"url": url}\n    headers = {\'Content-Type\': \'application/json\'}\n    \n    try:\n        response = l402_requests.post(endpoint, json=payload, headers=headers)\n        response.raise_for_status

In [274]:

# Step 4: Create the function
cf = create_func(extracted_code)
print("Function created successfully.")

Function created successfully.


In [256]:
# function('https://www.spacex.com/')

{'success': True,
 'data': {'markdown': "UPCOMING LAUNCH\n\n## Starship's Fifth Flight Test\n\n[LEARN MORE](/launches/mission/?missionId=starship-flight-5)\n\nRecent Launch\n\n## Hera Mission\n\n[Rewatch](/launches/mission/?missionId=hera)\n\nCURRENT MISSION\n\n## CREW-9 MISSION\n\n[REWATCH](/launches/mission/?missionId=crew-9)\n\nRECENT MISSION\n\n## POLARIS DAWN\n\n[REWATCH](/launches/mission/?missionId=polarisdawn)\n\n## ADVANCING HUMAN SPACEFLIGHT\n\n[LEARN MORE](/updates/#eva-suit)\n\n## TO MAKE LIFE MULTIPLANETARY\n\n[LEARN MORE](/updates/#make-life-multiplanetary)",
  'metadata': {'title': 'SpaceXSpaceX Logo',
   'description': 'SpaceX designs, manufactures and launches advanced rockets and spacecraft. The company was founded in 2002 to revolutionize space technology, with the ultimate goal of enabling people to live on other planets.',
   'language': 'en',
   'keywords': 'space,spacex,aerospace,elon musk,mars,falcon 9,falcon heavy,dragon',
   'ogTitle': 'SpaceX',
   'ogDescript

In [275]:
tools = [get_l402_uri_info, cf]

In [276]:
chat = Chat('gpt-4o-mini', sp='You are a helpful assistant that provides information about L402 URIs and your available tools.', tools=tools)


In [263]:
chat("which tools do you have available?")

I have the following tools available:

1. **get_l402_uri_info**: This tool returns information about a specific L402 URI resource. You provide the URI, and it returns a description of that resource.

2. **scrape_web_url**: This tool scrapes a given URL by sending a POST request to the specified endpoint with the URL included in the request body. It returns the response data from the web scraping request.

If you need to use either of these tools, just let me know how I can assist you!

<details>

- id: chatcmpl-AGTn00jXLrUVjkDzdQfT0RJcIFYAZ
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I have the following tools available:\n\n1. **get_l402_uri_info**: This tool returns information about a specific L402 URI resource. You provide the URI, and it returns a description of that resource.\n\n2. **scrape_web_url**: This tool scrapes a given URL by sending a POST request to the specified endpoint with the URL included in the request body. It returns the response data from the web scraping request.\n\nIf you need to use either of these tools, just let me know how I can assist you!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728491122
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_f85bea6784
- usage: CompletionUsage(completion_tokens=109, prompt_tokens=189, total_tokens=298, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [278]:
chat.toolloop("can you scrape this url for me: https://www.spacex.com/")


The scraping of the URL "https://www.spacex.com/" has returned the following content and metadata:

### Markdown Content:
- **UPCOMING LAUNCH**
  - **Starship's Fifth Flight Test**  
    [LEARN MORE](/launches/mission/?missionId=starship-flight-5)

- **Recent Launch**
  - **Hera Mission**  
    [Rewatch](/launches/mission/?missionId=hera)

- **CURRENT MISSION**
  - **CREW-9 MISSION**  
    [REWATCH](/launches/mission/?missionId=crew-9)

- **RECENT MISSION**
  - **POLARIS DAWN**  
    [REWATCH](/launches/mission/?missionId=polarisdawn)

- **ADVANCING HUMAN SPACEFLIGHT**  
  [LEARN MORE](/updates/#eva-suit)

- **TO MAKE LIFE MULTIPLANETARY**  
  [LEARN MORE](/updates/#make-life-multiplanetary)

### Metadata:
- **Title:** SpaceX | SpaceX Logo
- **Description:** SpaceX designs, manufactures and launches advanced rockets and spacecraft. The company was founded in 2002 to revolutionize space technology, with the ultimate goal of enabling people to live on other planets.
- **Language:** English (en)
- **Keywords:** space, spacex, aerospace, elon musk, mars, falcon 9, falcon heavy, dragon
- **Open Graph Title:** SpaceX
- **Open Graph Description:** SpaceX designs, manufactures and launches advanced rockets and spacecraft.
- **Open Graph URL:** http://www.spacex.com
- **Open Graph Image:** ![Image](https://www.spacex.com/static/images/share.jpg)
- **Open Graph Locale:** en_US
- **Source URL:** https://www.spacex.com/
- **Status Code:** 200 (indicating that the request was successful)

If you need further details or specific information, feel free to ask!

<details>

- id: chatcmpl-AGTqxkjuqPbT6HHkEO5EWFpHrDHVq
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The scraping of the URL "https://www.spacex.com/" has returned the following content and metadata:\n\n### Markdown Content:\n- **UPCOMING LAUNCH**\n  - **Starship\'s Fifth Flight Test**  \n    [LEARN MORE](/launches/mission/?missionId=starship-flight-5)\n\n- **Recent Launch**\n  - **Hera Mission**  \n    [Rewatch](/launches/mission/?missionId=hera)\n\n- **CURRENT MISSION**\n  - **CREW-9 MISSION**  \n    [REWATCH](/launches/mission/?missionId=crew-9)\n\n- **RECENT MISSION**\n  - **POLARIS DAWN**  \n    [REWATCH](/launches/mission/?missionId=polarisdawn)\n\n- **ADVANCING HUMAN SPACEFLIGHT**  \n  [LEARN MORE](/updates/#eva-suit)\n\n- **TO MAKE LIFE MULTIPLANETARY**  \n  [LEARN MORE](/updates/#make-life-multiplanetary)\n\n### Metadata:\n- **Title:** SpaceX | SpaceX Logo\n- **Description:** SpaceX designs, manufactures and launches advanced rockets and spacecraft. The company was founded in 2002 to revolutionize space technology, with the ultimate goal of enabling people to live on other planets.\n- **Language:** English (en)\n- **Keywords:** space, spacex, aerospace, elon musk, mars, falcon 9, falcon heavy, dragon\n- **Open Graph Title:** SpaceX\n- **Open Graph Description:** SpaceX designs, manufactures and launches advanced rockets and spacecraft.\n- **Open Graph URL:** http://www.spacex.com\n- **Open Graph Image:** ![Image](https://www.spacex.com/static/images/share.jpg)\n- **Open Graph Locale:** en_US\n- **Source URL:** https://www.spacex.com/\n- **Status Code:** 200 (indicating that the request was successful)\n\nIf you need further details or specific information, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728491367
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=427, prompt_tokens=783, total_tokens=1210, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>